# Keras Sequential LSTM ANN for Market Trading

## <i> i.) Import Libraries and Dependencies </i>

In [1]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
import pandas_ta as ta

from matplotlib import pyplot as plt
from pathlib import Path
import os
import re

In [2]:
# Import Keras ANN Modeling Libraries & Sklearn Preprocessing Modules
from keras.models import Sequential, Model
from keras.layers import LSTM
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pandas.tseries.offsets import DateOffset

# Import Keras ANN Dropout & Regularizer Build Processing Modules
# Import dropout
from keras.layers import Dropout
# Import regularizers
from keras.regularizers import l1, l2

2022-12-06 17:42:22.945259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## <i> ii.) Prepare the DataFrames </i>

In [3]:
# Get filename from dir and store as 'algoData_asset' for each individual asset file to use later in labelling/filenaming
filename = Path("../Datasets/algoData_S&P500.csv")

display(os.path.basename(filename))
display(os.path.dirname(filename))
display(os.path.splitext(filename))
display(os.path.splitext(os.path.basename(filename)))
algoData_asset = os.path.splitext(os.path.basename(filename))
algoData_asset = algoData_asset[0] 
algoData_asset = algoData_asset.split('_')
algoData_asset = algoData_asset[1]
print(algoData_asset)

#print(os.path.splitext(filename))

'algoData_S&P500.csv'

'../Datasets'

('../Datasets/algoData_S&P500', '.csv')

('algoData_S&P500', '.csv')

S&P500


In [4]:
# Confirm algoData_asset is type(str)
type(algoData_asset)

str

In [5]:
#Import .csv file as dataframe and shift 'ActualReturns' column forward .shift() 
#to re-align to neutral (as the 'Signal' column and 'ActualReturns' are both shifted backed (.shift(-1)) in the raw .csv file

# Import filename & conver to dataframe
df = pd.read_csv(filename, 
                index_col='Date', 
                parse_dates=True,
                infer_datetime_format=True)
display(df.head(3))
display(df.tail(3))

# Create empty dataframe for 'ActualReturns' column and shift(+1) to realign to neutral timeseries index position
actual_returns_shift_df = pd.DataFrame()
actual_returns_shift_df['Norm Actual Returns'] = df['ActualReturns'].shift()

,Ticker,Date.1,Open,High,Low,Close,Volume,ActualReturns,Signal,EMAShort,...,MOMline_normal,ROCline_normal,SMAline_normal,lowerBB_normal,middleBB_normal,upperBB_normal,2stdBB_normal,MACDline_normal,MACDHistogram_normal,MACDSignal_normal
Date,,,,,,,,,,,,,,,,,,,,,
2006-10-23 00:00:00-04:00,S&P500,2006-10-23 00:00:00-04:00,1368.579956,1377.400024,1363.939941,1377.020020,2480430000,0.000261,1.0,1361.879276,...,7.326168,0.808229,2.205601,1.057307,1.026343,0.997142,0.004253,14.443024,0.524587,13.918438
2006-10-24 00:00:00-04:00,S&P500,2006-10-24 00:00:00-04:00,1377.020020,1377.780029,1372.420044,1377.380005,2876890000,0.003514,1.0,1364.264003,...,4.586124,0.648913,2.183401,1.056273,1.024972,0.995474,0.004303,14.609685,0.552998,14.056687
2006-10-25 00:00:00-04:00,S&P500,2006-10-25 00:00:00-04:00,1377.359985,1383.609985,1376.000000,1382.219971,2953540000,0.004963,1.0,1367.026460,...,4.767735,0.564316,1.932800,1.059152,1.026940,0.996630,0.004401,14.959862,0.722540,14.237322


,Ticker,Date.1,Open,High,Low,Close,Volume,ActualReturns,Signal,EMAShort,...,MOMline_normal,ROCline_normal,SMAline_normal,lowerBB_normal,middleBB_normal,upperBB_normal,2stdBB_normal,MACDline_normal,MACDHistogram_normal,MACDSignal_normal
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-26 00:00:00-04:00,S&P500,2022-10-26 00:00:00-04:00,3825.969971,3886.149902,3824.070068,3830.600098,4817310000,-0.006083,-1.0,3743.064400,...,8.289918,0.484020,-9.492002,1.085286,1.028940,0.978155,0.002711,-9.503818,34.608016,-44.111835
2022-10-27 00:00:00-04:00,S&P500,2022-10-27 00:00:00-04:00,3834.689941,3859.949951,3803.790039,3807.300049,4687320000,0.024626,1.0,3752.946808,...,16.903382,0.239292,-9.334800,1.076946,1.023543,0.975186,0.002605,-3.675620,32.348971,-36.024592
2022-10-28 00:00:00-04:00,S&P500,2022-10-28 00:00:00-04:00,3808.260010,3905.419922,3808.260010,3901.060059,4459410000,-0.007454,-1.0,3775.733462,...,28.599808,0.709188,-7.653604,1.104215,1.048489,0.998117,0.002587,8.411950,35.549233,-27.137283


In [6]:
actual_returns_shift_df = actual_returns_shift_df.dropna()

In [7]:
display(actual_returns_shift_df)

,Norm Actual Returns
Date,
2006-10-24 00:00:00-04:00,0.000261
2006-10-25 00:00:00-04:00,0.003514
2006-10-26 00:00:00-04:00,0.004963
2006-10-27 00:00:00-04:00,-0.008452
2006-10-30 00:00:00-05:00,0.000428
...,...
2022-10-24 00:00:00-04:00,0.011882
2022-10-25 00:00:00-04:00,0.016267
2022-10-26 00:00:00-04:00,-0.007388


In [8]:
# Create list for non-normalized columns to be dropped out if required
%pprint
col_names = list(df.columns.values)
display(col_names)
display(f"Number of unique columns: {len(col_names)}")

Pretty printing has been turned OFF


['Ticker', 'Date.1', 'Open', 'High', 'Low', 'Close', 'Volume', 'ActualReturns', 'Signal', 'EMAShort', 'EMALong', 'RSIline', 'MOMline', 'ROCline', 'SMAline', 'lowerBB', 'middleBB', 'upperBB', '2stdBB', 'MACDline', 'MACDHistogram', 'MACDSignal', 'EMAShort_zscore', 'EMALong_zscore', 'RSIline_zscore', 'MOMline_zscore', 'ROCline_zscore', 'SMAline_zscore', 'lowerBB_zscore', 'middleBB_zscore', 'upperBB_zscore', '2stdBB_zscore', 'MACDline_zscore', 'MACDHistogram_zscore', 'MACDSignal_zscore', 'EMAShort_normal', 'EMALong_normal', 'RSIline_normal', 'MOMline_normal', 'ROCline_normal', 'SMAline_normal', 'lowerBB_normal', 'middleBB_normal', 'upperBB_normal', '2stdBB_normal', 'MACDline_normal', 'MACDHistogram_normal', 'MACDSignal_normal']

'Number of unique columns: 48'

## <i> ii.) Split Data into Training & Test Sets for LSTM </i>

In [9]:
# Create X features dataframe for LSTM
# Drop non-normalized & pre-Zscore calculated column values
# Drop all Bear & Bull signals for Elder Ray as they do not apply to forex strategy only
# I.e. 'lowerBB_normal', 'middleBB_normal', 'upperBB_normal', '2stdBB_normal'

must_drop_feat = ['Ticker', 'Date.1']

OHLCV = ['Open', 'High', 'Low', 'Close', 'Volume'] 

standard_feat = ['EMAShort', 'EMALong', 'RSIline', 'MOMline', 'ROCline', 'SMAline', 'lowerBB', \
                 'middleBB', 'upperBB', '2stdBB', 'MACDline', 'MACDHistogram', 'MACDSignal']

normal_feat = ['EMAShort_normal', 'EMALong_normal', 'RSIline_normal', 'MOMline_normal', 'ROCline_normal', \
               'SMAline_normal', 'lowerBB_normal', 'middleBB_normal', 'upperBB_normal', \
               '2stdBB_normal', 'MACDline_normal', 'MACDHistogram_normal', 'MACDSignal_normal']

zscore_feat = ['EMAShort_zscore', 'EMALong_zscore', 'RSIline_zscore', 'MOMline_zscore', 'ROCline_zscore', \
               'SMAline_zscore', 'lowerBB_zscore', 'middleBB_zscore', 'upperBB_zscore', \
               '2stdBB_zscore', 'MACDline_zscore', 'MACDHistogram_zscore', 'MACDSignal_zscore']

oscillator_feat = ['RSIline', 'MOMline', 'ROCline','MACDline', 'MACDHistogram', 'MACDSignal']

oscillator_wo_RSI_feat = ['MOMline', 'ROCline','MACDline', 'MACDHistogram', 'MACDSignal']

# Choose backtest variable to be used later on to cut y_train & y_test prematurely
backcandles = 30

X = df.copy()
X = X.drop(columns = zscore_feat)
#X = X.drop(columns = standard_feat)
#X = X.drop(columns = OHLCV)
X = X.drop(columns = must_drop_feat)
#X = X.drop(columns = normal_feat)
#X = X.drop(columns = oscillator_feat)
X = X.drop(columns = oscillator_wo_RSI_feat)
X['ActualReturns'] = actual_returns_shift_df['Norm Actual Returns']
X = X.dropna()
#y = X[['Signal']]
X = X.drop(columns = ['Signal'])
X = X.drop(columns = ['Volume'])
X = X.drop(columns = ['SMAline'])

# Create target feature which is the difference between the open/close of the day and shifted back (-1) day(s) in order to predict
X['Target_Diff'] = X['Close'] - X['Open']
X['Target_Diff'] = X['Target_Diff'].shift(-1)

# Create feature target class that displays if close>open=1 & close<open=0 (1,0 for sigmoid compatibility)
X['Target_Class'] = [1 if X['Target_Diff'][i] > 0 else 0 for i in range(len(X))]

# Create feature target next close and shift back one day in order to predict for the next day
X['Target_Next_Close'] = X['Close'].shift(-1)

#X = X.reindex(columns=[0, 1, 2, 3, 'Target',])
X = X[['Open', 'High', 'Low', 'Close', 'RSIline', 'EMAShort', 'EMALong', 'Target_Diff', 'Target_Class', 'Target_Next_Close']]

# Set prediction column -> in this case try and predict buy/sell signal (1,0) with Target_Class
y = X[['Target_Next_Close']]

display(f"Number of included columns in final X-features: {len(X.columns)}")
display(X)
display(y)

'Number of included columns in final X-features: 10'

,Open,High,Low,Close,RSIline,EMAShort,EMALong,Target_Diff,Target_Class,Target_Next_Close
Date,,,,,,,,,,
2006-10-24 00:00:00-04:00,1377.020020,1377.780029,1372.420044,1377.380005,75.281252,1364.264003,1349.654434,4.859985,1,1382.219971
2006-10-25 00:00:00-04:00,1377.359985,1383.609985,1376.000000,1382.219971,77.239463,1367.026460,1352.066696,6.869995,1,1389.079956
2006-10-26 00:00:00-04:00,1382.209961,1389.449951,1379.469971,1389.079956,79.694774,1370.419305,1354.808419,-11.550049,0,1377.339966
2006-10-27 00:00:00-04:00,1388.890015,1388.890015,1375.849976,1377.339966,66.477855,1371.484022,1356.477422,0.630005,1,1377.930054
2006-10-30 00:00:00-05:00,1377.300049,1381.219971,1373.459961,1377.930054,66.776108,1372.475720,1358.066506,0.009888,1,1377.939941
...,...,...,...,...,...,...,...,...,...,...
2022-10-24 00:00:00-04:00,3762.010010,3810.739990,3741.649902,3797.340088,54.033681,3703.155857,3737.302913,59.670166,1,3859.110107
2022-10-25 00:00:00-04:00,3799.439941,3862.850098,3799.439941,3859.110107,57.765246,3727.148819,3746.325668,4.630127,1,3830.600098
2022-10-26 00:00:00-04:00,3825.969971,3886.149902,3824.070068,3830.600098,55.524760,3743.064400,3752.568218,-27.389893,0,3807.300049


,Target_Next_Close
Date,
2006-10-24 00:00:00-04:00,1382.219971
2006-10-25 00:00:00-04:00,1389.079956
2006-10-26 00:00:00-04:00,1377.339966
2006-10-27 00:00:00-04:00,1377.930054
2006-10-30 00:00:00-05:00,1377.939941
...,...
2022-10-24 00:00:00-04:00,3859.110107
2022-10-25 00:00:00-04:00,3830.600098
2022-10-26 00:00:00-04:00,3807.300049


In [10]:
# Splitting Datasets Using 145 Week Groupings
# Select trading begin
training_begin = X.index.min()
print(f"Training begin: {training_begin}.")

# Select trading end
# Note: months = 145 for all assets except ETH-USD (months= 14)
# Note: months = 145 for all assets except BTC-USD (months= 70)
training_end = X.index.min() + DateOffset(months = 145)
print(f"Training end: {training_end}.")

X_train = X.loc[training_begin:training_end] 
y_train = y.loc[training_begin:training_end] 

X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Slice y_train & y_test by backcandle requirements
#y_train = y_train_1.iloc[backcandles:]
#y_test = y_test_1.iloc[backcandles:]

# Create Dataframe just to hold 'ActualReturns' values for end of calculations
X_ActualReturns = actual_returns_shift_df.loc[training_end:]
                
#Also save a second X_test_aligned_index for final dataframe
X_test_aligned_index = X.loc[training_end:]

Training begin: 2006-10-24 00:00:00-04:00.
Training end: 2018-11-24 00:00:00-04:00.


In [11]:
# Test output y_train
X_train

,Open,High,Low,Close,RSIline,EMAShort,EMALong,Target_Diff,Target_Class,Target_Next_Close
Date,,,,,,,,,,
2006-10-24 00:00:00-04:00,1377.020020,1377.780029,1372.420044,1377.380005,75.281252,1364.264003,1349.654434,4.859985,1,1382.219971
2006-10-25 00:00:00-04:00,1377.359985,1383.609985,1376.000000,1382.219971,77.239463,1367.026460,1352.066696,6.869995,1,1389.079956
2006-10-26 00:00:00-04:00,1382.209961,1389.449951,1379.469971,1389.079956,79.694774,1370.419305,1354.808419,-11.550049,0,1377.339966
2006-10-27 00:00:00-04:00,1388.890015,1388.890015,1375.849976,1377.339966,66.477855,1371.484022,1356.477422,0.630005,1,1377.930054
2006-10-30 00:00:00-05:00,1377.300049,1381.219971,1373.459961,1377.930054,66.776108,1372.475720,1358.066506,0.009888,1,1377.939941
...,...,...,...,...,...,...,...,...,...,...
2018-11-16 00:00:00-05:00,2718.540039,2746.750000,2712.159912,2736.270020,46.746647,2738.640616,2755.921907,-40.010010,0,2690.729980
2018-11-19 00:00:00-05:00,2730.739990,2733.159912,2681.090088,2690.729980,40.997283,2731.269749,2751.092875,-12.710205,0,2641.889893
2018-11-20 00:00:00-05:00,2654.600098,2669.439941,2631.520020,2641.889893,35.898018,2717.519002,2743.003765,-7.810059,0,2649.929932


In [12]:
y_train

,Target_Next_Close
Date,
2006-10-24 00:00:00-04:00,1382.219971
2006-10-25 00:00:00-04:00,1389.079956
2006-10-26 00:00:00-04:00,1377.339966
2006-10-27 00:00:00-04:00,1377.930054
2006-10-30 00:00:00-05:00,1377.939941
...,...
2018-11-16 00:00:00-05:00,2690.729980
2018-11-19 00:00:00-05:00,2641.889893
2018-11-20 00:00:00-05:00,2649.929932


In [13]:
#backcandles_offset_start_train = y_train.index.min() + DateOffset(days = backcandles)

#y_train = y_train.loc[backcandles_offset_start_train:]
#y_train

In [14]:
# y_test test
y_test

,Target_Next_Close
Date,
2018-11-26 00:00:00-05:00,2682.169922
2018-11-27 00:00:00-05:00,2743.790039
2018-11-28 00:00:00-05:00,2737.800049
2018-11-29 00:00:00-05:00,2760.169922
2018-11-30 00:00:00-05:00,2790.370117
...,...
2022-10-24 00:00:00-04:00,3859.110107
2022-10-25 00:00:00-04:00,3830.600098
2022-10-26 00:00:00-04:00,3807.300049


In [15]:
X_test

,Open,High,Low,Close,RSIline,EMAShort,EMALong,Target_Diff,Target_Class,Target_Next_Close
Date,,,,,,,,,,
2018-11-26 00:00:00-05:00,2649.969971,2674.350098,2649.969971,2673.449951,42.470118,2692.234454,2724.365816,18.419922,1,2682.169922
2018-11-27 00:00:00-05:00,2663.750000,2682.530029,2655.889893,2682.169922,43.863468,2690.686064,2721.240194,52.340088,1,2743.790039
2018-11-28 00:00:00-05:00,2691.449951,2744.000000,2684.379883,2743.790039,52.599970,2698.855907,2722.910553,0.830078,1,2737.800049
2018-11-29 00:00:00-05:00,2736.969971,2753.750000,2722.939941,2737.800049,51.756738,2704.847313,2724.013479,22.409912,1,2760.169922
2018-11-30 00:00:00-05:00,2737.760010,2760.879883,2732.760010,2760.169922,54.678773,2713.358484,2726.691734,-0.129883,0,2790.370117
...,...,...,...,...,...,...,...,...,...,...
2022-10-24 00:00:00-04:00,3762.010010,3810.739990,3741.649902,3797.340088,54.033681,3703.155857,3737.302913,59.670166,1,3859.110107
2022-10-25 00:00:00-04:00,3799.439941,3862.850098,3799.439941,3859.110107,57.765246,3727.148819,3746.325668,4.630127,1,3830.600098
2022-10-26 00:00:00-04:00,3825.969971,3886.149902,3824.070068,3830.600098,55.524760,3743.064400,3752.568218,-27.389893,0,3807.300049


In [16]:
# Test output y_test
#backcandles_offset_start_test = y_test.index.min() + DateOffset(days = backcandles)

#y_test = y_test.loc[backcandles_offset_start_test:]

In [17]:
#y_test

In [18]:
# Function to convert all sell signals (-1) to be compatible with the output sigmoid activation function

#def convert_neg(y_df):
#    for i in (range(len(y_df))):
#        if (y_df['Signal'][i] == -1):
#            y_df['Signal'][i] = 0
#        
#    return y_df

In [19]:
#y_train.copy()
#y_train = convert_neg(y_train)

In [20]:
#y_test.copy()
#y_test = convert_neg(y_test)

In [21]:
# Re-test output y_train after -1->0 conversion for sigmoid output compatiblity 
#y_train

In [35]:
# Training & Test Set Dimensions

display(f"X Dataframe shape: {X.shape}.")
display(f"y Dataframe shape: {y.shape}.")
display(f"X_train Dataframe shape: {X_train.shape}.")
display(f"y_train shape: {y_train.shape}.")
display(f"X_test Dataframe shape: {X_test.shape}.")
display(f"y_test shape: {y_test.shape}.")

'X Dataframe shape: (4032, 10).'

'y Dataframe shape: (4032, 1).'

'X_train Dataframe shape: (3043, 10).'

'y_train shape: (3043, 1).'

'X_test Dataframe shape: (989, 10).'

'y_test shape: (989, 1).'

In [45]:
# Convert y_train/y_test dataframe ['Target_Class'] columns values to array for Keras Sequential fitting 

#y_train = y_train['Target_Next_Close'].values
display(y_train[0:3])
display(y_train[-1])
#y_test = y_test['Target_Next_Close'].values
display(y_test[0:3])
display(y_test[-2])

array([[1382.2199707 ],
       [1389.07995605],
       [1377.33996582]])

array([2673.44995117])

array([[2682.16992188],
       [2743.79003906],
       [2737.80004883]])

array([3901.06005859])

In [24]:
# Finalize the y_train & y_test data into a trainable array
# Reshape into vertical vectors for use in Sequential Model algorithm

#y_train = y_train['Signal'].values.astype("int32")
#y_train = y_train.reshape(-1,1).astype("int32")
y_train = y_train.reshape(-1,1)

#y_test = y_test['Signal'].values.astype("int32")
y_test = y_test.reshape(-1,1)

display((f"Number of final X_test features to run into ANN: {len(X_test.columns)}."))
display(y_train[0:10])
display(y_test[0:10])

'Number of final X_test features to run into ANN: 10.'

array([[1382.2199707 ],
       [1389.07995605],
       [1377.33996582],
       [1377.93005371],
       [1377.93994141],
       [1367.81005859],
       [1367.33996582],
       [1364.30004883],
       [1379.7800293 ],
       [1382.83996582]])

array([[2682.16992188],
       [2743.79003906],
       [2737.80004883],
       [2760.16992188],
       [2790.37011719],
       [2700.06005859],
       [2695.94995117],
       [2633.08007812],
       [2637.7199707 ],
       [2636.7800293 ]])

In [25]:
# Create the StandardScaler instance
X_scaler = MinMaxScaler(feature_range=(0,1))
Y_scaler = MinMaxScaler(feature_range=(0,1))
#X_train_scaled = pd.DataFrame(X_scaler.fit_transform(X_train),columns =X_train.columns)
#X_test_scaled = pd.DataFrame(X_scaler.fit_transform(X_test),columns =X_test.columns)

# Fit the scaler
X_scaler.fit(X_train)
Y_scaler.fit(y_train)

# Scale the X_train & X_test data 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Scale the y_train & y_test data
y_train_scaled = Y_scaler.transform(y_train)
y_test_scaled = Y_scaler.transform(y_test)



# Display X_train_scaled & X_test_scaled shapes
display(f"X_train_scaled shape: {X_train_scaled.shape}")
display(f"X_train_scaled shape: {X_test_scaled.shape}")
display(f"y_train_scaled shape: {y_train_scaled.shape}")
display(f"y_test_scaled shape: {y_test_scaled.shape}")

'X_train_scaled shape: (3043, 10)'

'X_train_scaled shape: (989, 10)'

'y_train_scaled shape: (3043, 1)'

'y_test_scaled shape: (989, 1)'

In [26]:
# Backtesting Setup for Training Data
# x = list of the number of features to be provided from the data to the model
x_train_scaled = []

# Setting backcandles for backtesting/short-term memory recursive setup
# Backcandles 10 candles (closing prices) back in time to predict the next days
# where j = number of X features to be included in test
# where i = number the backcandles grouping represents through all index rows 
print(f'X_train_scaled shape: {X_train_scaled.shape[0]}.')
for j in range((X_train_scaled.shape[1]-3)): # range is the number of input columns counting from 1->j[-1], i.e in this case range = 11
    x_train_scaled.append([])
    for i in range(backcandles, X_train_scaled.shape[0]):
        x_train_scaled[j].append(X_train_scaled[i-backcandles:i,j])
        
x_train_scaled=np.moveaxis(x_train_scaled, [0], [2])

x_train_scaled = np.array(x_train_scaled)


y_train_scaled = y_train_scaled[backcandles:]


#*** New

print()
print("'X_train_scaled' --> transformed into 'x_trained_scaled' ")
print(f'x_trained_scaled shape: {x_train_scaled.shape}')
print(f'y_train shape: {y_train_scaled.shape}')

X_train_scaled shape: 3043.

'X_train_scaled' --> transformed into 'x_trained_scaled' 
x_trained_scaled shape: (3013, 30, 7)
y_train shape: (3013, 1)


In [27]:
# Backtesting Setup for Testing Data
# x = list of the number of features to be provided from the data to the model
x_test_scaled = []

# Setting backcandles for backtesting/short-term memory recursive setup
# Backcandles 10 candles (closing prices) back in time to predict the next days
# where j = number of X features to be included in test
# where i = number the backcandles grouping represents through all index rows 
print(X_test_scaled.shape[0])
for j in range((X_test_scaled.shape[1]-3)): # range is the number of input columns counting from 1->j[-1], i.e in this case range = 11
    x_test_scaled.append([])
    for i in range(backcandles, X_test_scaled.shape[0]):
        x_test_scaled[j].append(X_test_scaled[i-backcandles:i,j])
        
x_test_scaled=np.moveaxis(x_test_scaled, [0], [2])

x_test_scaled = np.array(x_test_scaled)

y_test_scaled = y_test_scaled[backcandles:]

print()
print("'X_test_scaled' --> transformed into 'x_test_scaled' ")
print(f'x_test_scaled shape: {x_test_scaled.shape}')
print(f'y_test_scaled shape: {y_test_scaled.shape}')

989

'X_test_scaled' --> transformed into 'x_test_scaled' 
x_test_scaled shape: (959, 30, 7)
y_test_scaled shape: (959, 1)


## <i> iii.) Constructing the LSTM Model Using Sequential Model & LSTM Module</i>

### <i> Defining the Input Parameters, Compile & Fit LSTM: </i>

In [29]:
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate

# Instantiate the Sequential Model()
nn = Sequential()
# Add first Dense Layer
# Add the LSTM time-dependent window layers

nn.add(LSTM(units=8, return_sequences=True, input_shape=(backcandles, (X_test_scaled.shape[1]-3))))
# Add first dropout
nn.add(Dropout(0.1))
# Add second LSTM time-dependent window layers
nn.add(LSTM(units=8, return_sequences=True))
# Add second dropout
nn.add(Dropout(0.1))
# Add third LSTM time-dependent window layers
nn.add(LSTM(units=8, return_sequences=True))
# Add third dropout
nn.add(Dropout(0.1))
# Add fourth LSTM time-dependent window layers
nn.add(LSTM(units=8))
# Add fourth dropout
nn.add(Dropout(0.1))
# Add output layer
nn.add(Dense(units=1))
# Output LSTM summary statistics

# Compile LSTM Model
nn.compile(optimizer='adam', loss='mse', metrics=["accuracy"])
# Fit LSTM Model
fit_model = nn.fit(x=x_train_scaled, y=y_train_scaled, batch_size=128, epochs=50, verbose=2)

Epoch 1/50
24/24 - 9s - loss: 0.1610 - accuracy: 3.3190e-04 - 9s/epoch - 363ms/step
Epoch 2/50
24/24 - 1s - loss: 0.0370 - accuracy: 6.6379e-04 - 1s/epoch - 54ms/step
Epoch 3/50
24/24 - 1s - loss: 0.0127 - accuracy: 6.6379e-04 - 1s/epoch - 52ms/step
Epoch 4/50
24/24 - 1s - loss: 0.0094 - accuracy: 6.6379e-04 - 1s/epoch - 54ms/step
Epoch 5/50
24/24 - 1s - loss: 0.0084 - accuracy: 6.6379e-04 - 1s/epoch - 51ms/step
Epoch 6/50
24/24 - 1s - loss: 0.0078 - accuracy: 6.6379e-04 - 1s/epoch - 50ms/step
Epoch 7/50
24/24 - 1s - loss: 0.0073 - accuracy: 6.6379e-04 - 1s/epoch - 50ms/step
Epoch 8/50
24/24 - 1s - loss: 0.0070 - accuracy: 6.6379e-04 - 1s/epoch - 61ms/step
Epoch 9/50
24/24 - 1s - loss: 0.0066 - accuracy: 6.6379e-04 - 1s/epoch - 53ms/step
Epoch 10/50
24/24 - 1s - loss: 0.0067 - accuracy: 6.6379e-04 - 1s/epoch - 53ms/step
Epoch 11/50
24/24 - 1s - loss: 0.0057 - accuracy: 6.6379e-04 - 1s/epoch - 49ms/step
Epoch 12/50
24/24 - 1s - loss: 0.0057 - accuracy: 6.6379e-04 - 1s/epoch - 49ms/step


KeyboardInterrupt: 

In [ ]:
# Plot the Output of the fit_model history
fig, ax = plt.subplots(1) # Creates figure fig and add an axes, ax.
fig2, ax2 = plt.subplots(1) # Another figure

ax.set_title(f' ANN LSTM ((Linear Output Act.) - {algoData_asset} Accuracy Plot')
ax.plot(fit_model.history['accuracy'], label='train')
fig.savefig(f'../Datasets/algoData_LSTM_results/{algoData_asset}_LSTM_accuracy_.png')


ax2.set_title(f' ANN LSTM (Linear Output Act.) - {algoData_asset} Loss Plot')
ax2.plot(fit_model.history['loss'], label='train')
fig2.savefig(f'../Datasets/algoData_LSTM_results/{algoData_asset}_LSTM_loss_.png')

In [ ]:
# Evaluate the model using testing data
model_loss, model_accuracy = nn.evaluate(x_test_scaled, y_test, verbose=2)

# Display evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Predict on X_test, y_test datasets

print("Evaluate model on X_test_scaled data")
#predictions = nn.predict(X_test_scaled).astype("int32")
predictions = (nn.predict(x_test_scaled) > 0.5).astype("int32")
#predictions = nn.predict(x_test_scaled)

In [ ]:
display(predictions[0:40])

In [ ]:
y_test = y_test.flatten()

In [ ]:
results = pd.DataFrame({"predictions": predictions.flatten(), "actual": y_test.flatten()})
#results = pd.DataFrame({"predictions": predictions.ravel(), "actual": y_test}, index=[0])
display(results.value_counts())
display(results.head(40))
display(results.tail(40))

In [ ]:
# Function to convert all -1 to 0

#def convert_back_to_neg(y_df):
#    for i in (range(len(y_df))):
#        if (y_df['predictions'][i] == 0):
#            y_df['predictions'][i] = -1
#        
#    return y_df

In [ ]:
#results.copy()
#results = convert_back_to_neg(results)

In [ ]:
results

In [ ]:
results.value_counts()

In [ ]:
# Get the original Actual Returns column from the original imported dataset

df_final = df[['ActualReturns']]
df_final = df_final.loc[training_end:]
df_final

In [ ]:
# Create function to compile cumulative returns for Artificial Neural Network Predicted y_test values

def annReturns(results, X_test):

    # Make emtpy dataframe to store actual returns, predictions & cumulative returns
    
    # Inititialize first return 0 so cumulative ret begins at base 1
    #annData['Actual Returns'][0] = 0
    
    annData = pd.DataFrame()
    annData.index = X_test.index
    results.index = X_test.index
    annData['Prediction_Signal'] = results['predictions'] 
    annData['Actual_Returns'] = df_final['ActualReturns'].values
    annData['annStrategyReturns'] = annData['Actual_Returns'] * annData['Prediction_Signal'] 
    annData['cumActual_Returns'] = (1 + annData['Actual_Returns']).cumprod()
    annData['cumANNStrategyReturns'] = (1 + annData['annStrategyReturns']).cumprod()
    
    returns = annData
    
    return returns

In [ ]:
# Run annReturns Function and display sample final cumulative returns dataframe

returns_df = annReturns(results, X_test)
display(returns_df.head(5))
display(returns_df.tail(5))

In [ ]:
# Save Results to External .csv File
# Save Finalized Output 'returns_df' dataframe

returns_df.to_csv(f'../Datasets/algoData_LSTM_results/{algoData_asset}_ANN_results.csv', index=True)